In [15]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import os

In [ ]:
# data_path=r'D:\PythonDev\MyQuantFinProject\Data\thai-small_2023-now.csv'
data_path=r'D:\PythonDev\MyQuantFinProject\Data\TH-MyLike-07To09Y23.csv'
trade_plan_path=r'D:\PythonDev\MyQuantFinProject\FinQuant\TradeSimulation\data\Config-BuySellTrade.xlsx'

TotalBudget=100*2000

#trade_sheet='ThaiNov22-Now23'
trade_sheet='TH-MyLike-07To09Y23'

start_date='2023-07-01'
end_date='2023-09-30'



In [27]:
file_name=trade_sheet
file_name=f'THAI_{trade_sheet}'
noFee=False


fee_sheet='Fee'
# data_path=r'D:\PythonDev\MyQuantFinProject\Data\thai_fund.csv'

# excludeFund=['ASP-T12','KKP-TQG']
excludeFund=[]

if noFee==True:
 report_path=f'report/{file_name}(no-fee).xlsx'
else:
 report_path=f'report/{file_name}.xlsx' 
print(report_path)
# trade_sheet='BuySell_Buy_Hold'
# report_path=f'report/BuySell_4ETFFunds.xlsx'


report/THAI_TH-MyLike-07To09Y23.xlsx


In [ ]:
if os.path.exists(data_path) and  os.path.exists(data_path):
 
    print("Change paramter directly on BuySellPortfolio.py prior to running this file.")
    print(f"Filename : {data_path}")
    print(f"Trade Plan Sheet Name:{trade_sheet}")
    print(f"Trad config :{trade_plan_path}")
    print(f"Simulatite data from {start_date} to {end_date} on report {report_path}")
    press_y=input(f"Press y=True run trade buy/sell simulation by budget = {TotalBudget}/fund : ") 
    if press_y.lower()=='y':
     ok=True
    else:
     exit()
else:
    raise Exception(f"Not fould {data_path}  or {trade_plan_path}")

Change paramter directly on BuySellPortfolio.py prior to running this file.
Filename : D:\PythonDev\MyQuantFinProject\Data\TH-MyLike-07To09Y23.csv
Trade Plan Sheet Name:TH-MyLike-07To09Y23
Trad config :D:\PythonDev\MyQuantFinProject\FinQuant\TradeSimulation\data\Config-BuySellTrade.xlsx
Simulatite data from 2023-07-01 to 2023-09-30


Press y=True run trade buy/sell simulation by budget = 200000/fund :  y


# Read Price CSV

In [28]:
df=pd.read_csv(data_path,parse_dates=['Date/Time'],dayfirst=True)
df.columns=['symbol','date','price']
df=df.query('date>=@start_date and date<=@end_date')
df.info()
print(df.sample(10))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 496 entries, 0 to 495
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   symbol  496 non-null    object        
 1   date    496 non-null    datetime64[ns]
 2   price   496 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 15.5+ KB
         symbol       date  price
92      ASP-SME 2023-08-17  18.64
243  K-MIDSMALL 2023-09-25  16.76
309     KKP-TQG 2023-09-29   8.33
348  KKP-SM-CAP 2023-08-29  15.10
67      ASP-SME 2023-07-10  18.45
278     KKP-TQG 2023-08-17   8.15
363  KKP-SM-CAP 2023-09-19  14.91
414  TISCOFLEXP 2023-09-04  56.72
418  TISCOFLEXP 2023-09-08  57.14
203  K-MIDSMALL 2023-07-26  16.17


# List Symbol and Plot Chart

In [29]:
list_symbols=list(df['symbol'].unique())
list_symbols=[ f for f in list_symbols if f not in excludeFund ]
list_symbols
# list_symbols=['ACWI']


['ASP-T12',
 'ASP-SME',
 'ASP-THEQ',
 'K-MIDSMALL',
 'KKP-TQG',
 'KKP-SM-CAP',
 'TISCOFLEXP',
 'TSF-A']

In [30]:
# print(list_symbols)
# for  symbol in list_symbols:
#     print(f"=============================={symbol}==============================")
#     dfFund=df.query('symbol==@symbol')
#     plt.figure(figsize=(15, 8))
#     plt.plot(dfFund['date'],dfFund['price'])
#     plt.show()

# Get Trade Data and Fee

In [31]:
# read feeon 
dfAssetInfo=pd.read_excel(trade_plan_path,engine="openpyxl",sheet_name=fee_sheet,\
                    index_col='symbol', dtype ={'buy_fee_pct': np.float64,'sell_fee_pct': np.float64})

# check where  fund fee  in portfolio  are avaible?
dfNoFeeInfo= [ item for item in list_symbols if item not in list(dfAssetInfo.index)]
dfAssetInfo=dfAssetInfo.loc[list(list_symbols),:]

if noFee==True:
    dfAssetInfo[['buy_fee_pct','sell_fee_pct']]=0
    
print("fee info")
print(dfAssetInfo) 
# For instance   AAXJ Price=100 but we need to pay 101 $

fee info
            buy_fee_pct  sell_fee_pct      region
symbol                                           
ASP-T12            1.25           0.0        thai
ASP-SME            1.25           0.0  thai-small
ASP-THEQ           1.00           0.0        thai
K-MIDSMALL         1.00           0.0  thai-small
KKP-TQG            0.50           0.0  thai-small
KKP-SM-CAP         1.00           0.0  thai-small
TISCOFLEXP         1.00           0.0        thai
TSF-A              1.05           0.0        thai


In [32]:
dfPlan=pd.read_excel(trade_plan_path,engine="openpyxl",sheet_name=trade_sheet, \
                     parse_dates=['trade_date'],dtype ={'portion_pct': np.float64})

#total buy portion is equal or greater than sell portion
buy_sum_act=dfPlan.query("action=='b'").portion_pct.sum()
sell_sum_act=dfPlan.query("action=='s'").portion_pct.sum()

all_portion=0

In [33]:
if buy_sum_act<sell_sum_act:
  raise Exception("Not allow Buy <= Sell")   
else:
    if buy_sum_act>sell_sum_act:
       print("=============Calculate the last portion to be sellable")
       sellAllAtNow=buy_sum_act-sell_sum_act 
       dfLastPrice=pd.DataFrame(columns=list(dfPlan.columns), data=[ [df['date'].max(),sellAllAtNow,'s']])
       print(dfLastPrice)
       dfPlan=pd.concat([dfPlan,dfLastPrice])
       dfPlan=dfPlan.reset_index(drop=True)

    tradeDateList= [ d for d in list(dfPlan['trade_date']) ]    
    all_portion=buy_sum_act   
    print(f"portion = {all_portion} on")
    
print("==========================Summarize All Trade==========================================")
print(f"No Buy/Sell {dfPlan.groupby(['action']).size()}")
print(dfPlan)

portion = 100.0 on
==========================Summarize All Trade==========================================
No Buy/Sell action
b    4
s    4
dtype: int64
  trade_date  portion_pct action
0 2023-07-14         25.0      b
1 2023-08-07         25.0      b
2 2023-08-18         25.0      b
3 2023-08-22         25.0      b
4 2023-09-14         25.0      s
5 2023-09-18         25.0      s
6 2023-09-21         25.0      s
7 2023-09-28         25.0      s


In [34]:
# To make it easier for use to understand portfolio transatoin 
rrr=TotalBudget % all_portion
if rrr !=0 :
  raise Exception(f"Your Budget {TotalBudget} is  not proportional  Portion with {all_portion}")
else:
  print(f"Your Budget {TotalBudget} is  proportional  Portion with {all_portion}")

Your Budget 200000 is  proportional  Portion with 100.0


# Custom Function

In [35]:
# figure out the amount of qty from total fund total value
def get_qty_buy(item):
    xCost=(item['portion_pct']/all_portion)*TotalBudget  
    xPrice=item['price_on_fee']
    qty=xCost/xPrice
    return qty

# figure the amount of  qty out  from total qty
def get_qty_sell(item,TotalQty):  
    qty=(item['portion_pct']/all_portion)*TotalQty
    return qty

In [36]:
def set_fee_by_action(item):
  x=item['symbol']  
  if item['action']=='b':
    fee_type='buy_fee_pct'
  elif  item['action']=='s':
    fee_type='sell_fee_pct'
  try:
     return dfAssetInfo.loc[x,fee_type]
  except Exception as ex:
     print(f"Error : no {x} to get fund fee")
     return 0

In [37]:
dict_dfTran={}
dfSummary=pd.DataFrame(columns=['Symbol','BuyValue','AvgBuyPrice','SellValue','AvgSellPrice','PL','Percent_PL'])

dfSummary

,Symbol,BuyValue,AvgBuyPrice,SellValue,AvgSellPrice,PL,Percent_PL


In [16]:
#https://www.w3resource.com/pandas/dataframe/dataframe-apply.php

for symbol in list_symbols:
    print(f"*******************{symbol}*******************")

    dfBS=df[ (df['symbol']==symbol) & (df['date'].isin(tradeDateList))]
    # if len(tradeDateList)==len(dfBS):
        
     # Step1  join price data (filter by sybom and the date whiich you have buy/sell action)   
    dfBS=dfBS.merge(dfPlan,how='inner',left_on='date',right_on='trade_date')
     # remove  redundant data  
    dfBS=dfBS.drop(columns=['trade_date'])     
       
    
     # Step2 calculate fee function
    dfBS['fee']=dfBS.apply(set_fee_by_action,axis=1) # 2.1
    
     # The price that allows you to buy the fund is more expensive than the actual asset price  1%  ,other the other had 
     # Buy price*1.0x=??   
    dfBS['price_on_fee']=dfBS.apply(lambda item : item['price']*(1+ (item['fee']/100)) if item['action']=='b' else item['price']*(1-(item['fee']/100)), axis=1)
    dfBS['price_on_fee']=dfBS['price_on_fee'].round(2) #2.2   
    # print(dfBS)
        
     # Step3  To make it easier to calculate and trading sumary in tearm of qty both buy/sell , both use different way
     # we recommend spliting this dataframe  into 2 dataframe
     #  BuyDataFrame and Sell Date , and we will merge both later  
    
    #3.1 query only buy action
    print("======================Buy=======================")
    buyDF=dfBS.query("action=='b'")   
    
    # 3.2 send the  row to function to calculate qty 
    buyDF['qty']=buyDF.apply(get_qty_buy,axis=1) 
    buyDF['value']=round(buyDF['qty']*buyDF['price_on_fee'],0)
    
    
    # 1 is buy (I will take columne to apply cumsum function to calcaulate cumulative return
    buyDF['action_value']=buyDF['value']
    buyDF['action_qty']=buyDF['qty']
    
    # 3.3 Total qty and value and avg
    TotalQty=buyDF['qty'].sum()
    TotalValue=round(buyDF['value'].sum(),0)
    AvgPrice=round(TotalValue/TotalQty,2)
    
    # print(buyDF[['date','portion_pct','value' ,'price','fee','price_on_fee','qty']])
    
    
    # Step4 follow every step corresponidng to prev step#3
    print("=======================Sell=======================")
    # step#4.1
    sellDF=dfBS.query("action=='s'")  
    
    # ste#4.2
    sellDF['qty']=sellDF.apply(get_qty_sell,axis=1,args=(TotalQty,))
    sellDF['value']=round(sellDF['qty']*sellDF['price_on_fee'],0)
    
    # -1 is sell (I will take columne to apply cumsum function to calcaulate cumulative return
    sellDF['action_value']=sellDF['value']*-1
    sellDF['action_qty']=sellDF['qty']*-1
    
    # print(sellDF[['date' ,'portion_pct','value','price','fee','price_on_fee','qty']])
    

    
    print("======================Merge Buy & Sell======================")
    # step 5  concatenate buy and sell dataframe 
    dfTran=pd.concat([buyDF,sellDF],axis=0)
    dfTran=dfTran.sort_values(by=['date'])
    dfTran[['value','action_value']]=dfTran[['value','action_value']].round(0)

    # step 6 accumulate value and qury over entire data since start buying to last record 
    dfTran['net_value']=dfTran['action_value'].cumsum(axis = 0) 
    dfTran['net_qty']=dfTran['action_qty'].cumsum(axis = 0)  
    dfTran['avg_price']=round( dfTran['net_value']/dfTran['net_qty'],2)
    
    dfTran=dfTran.sort_values(by=['date'])
    # print(dfTran[['date' ,'action','price_on_fee','qty','net_qty','value','net_value','avg_price']])
    print(dfTran[['date' ,'action','price_on_fee','qty','value','avg_price']])
    
    dict_dfTran[symbol]=dfTran
    
    print("=======================Summarize Gain/Loss=======================")
    #step 7 create total buy/sell value , avg prie and profit/loss(value and %) of  all funds 
    # step 7.1 apply sum agg over  value and qty to find avg throughout  the whold dataframe
    SellTotalQty=sellDF['qty'].sum()
    SellTotalValue=round(sellDF['value'].sum(),0)
    SellAvgPrice=round(SellTotalValue/SellTotalQty,0)
    
    print(f"Total Buy {symbol} : qty={TotalQty} and value={TotalValue} at AvgPrice={AvgPrice}")
    print(f"Total Sell {symbol} : qty={SellTotalQty} and value={SellTotalValue} at AvgPrice={SellAvgPrice}")
    

    
    # step #7.2
    PL=round(SellTotalValue-TotalValue,0)
    PCT_PL=round(PL/TotalValue*100,2)
    
    # step# 7.3 
    # you take amont 100000.0 and gain profit103500 from investment on this etf fund 
    summary_data={'Symbol':symbol,'BuyValue':TotalValue,'AvgBuyPrice':AvgPrice,'SellValue':SellTotalValue,\
                  'AvgSellPrice':SellAvgPrice, 'PL':PL,'Percent_PL':PCT_PL
                 }
    dfSummary=dfSummary.append(summary_data,ignore_index=True)
    
    print(dfSummary)

    


*******************ASP-T12*******************
======================Buy=======================
=======================Sell=======================
======================Merge Buy & Sell======================
        date action  price_on_fee          qty    value  avg_price
0 2023-07-14      b         11.50  4347.826087  50000.0      11.50
1 2023-08-07      b         11.55  4329.004329  50000.0      11.52
2 2023-08-18      b         11.79  4240.882103  50000.0      11.61
3 2023-08-22      b         11.93  4191.114837  50000.0      11.69
4 2023-09-14      s         11.55  4277.206839  49402.0      11.74
5 2023-09-18      s         11.48  4277.206839  49102.0      11.86
6 2023-09-21      s         11.38  4277.206839  48675.0      12.35
7 2023-09-28      s         11.23  4277.206839  48033.0        inf
=======================Summarize Gain/Loss=======================
Total Buy ASP-T12 : qty=17108.827355984897 and value=200000.0 at AvgPrice=11.69
Total Sell ASP-T12 : qty=17108.827355984897 

C:\Users\pongthsa\AppData\Local\Temp\ipykernel_12372\186035005.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buyDF['qty']=buyDF.apply(get_qty_buy,axis=1)
C:\Users\pongthsa\AppData\Local\Temp\ipykernel_12372\186035005.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buyDF['value']=round(buyDF['qty']*buyDF['price_on_fee'],0)
C:\Users\pongthsa\AppData\Local\Temp\ipykernel_12372\186035005.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

======================Buy=======================
=======================Sell=======================
======================Merge Buy & Sell======================
        date action  price_on_fee         qty    value     avg_price
0 2023-07-14      b         75.36  663.481953  50000.0  7.536000e+01
1 2023-08-07      b         74.34  672.585418  50000.0  7.485000e+01
2 2023-08-18      b         75.72  660.327522  50000.0  7.514000e+01
3 2023-08-22      b         77.29  646.914219  50000.0  7.566000e+01
4 2023-09-14      s         75.73  660.827278  50044.0  7.564000e+01
5 2023-09-18      s         75.20  660.827278  49694.0  7.586000e+01
6 2023-09-21      s         75.10  660.827278  49628.0  7.662000e+01
7 2023-09-28      s         74.09  660.827278  48961.0  7.357932e+15
=======================Summarize Gain/Loss=======================
Total Buy TSF-A : qty=2643.3091132646737 and value=200000.0 at AvgPrice=75.66
Total Sell TSF-A : qty=2643.3091132646737 and value=198327.0 at AvgPrice=7

C:\Users\pongthsa\AppData\Local\Temp\ipykernel_12372\186035005.py:104: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfSummary=dfSummary.append(summary_data,ignore_index=True)


In [17]:
dfSummary

,Symbol,BuyValue,AvgBuyPrice,SellValue,AvgSellPrice,PL,Percent_PL
0,ASP-T12,200000.0,11.69,195212.0,11.0,-4788.0,-2.39
1,ASP-SME,200000.0,18.95,197558.0,19.0,-2442.0,-1.22
2,ASP-THEQ,200000.0,15.05,199147.0,15.0,-853.0,-0.43
3,K-MIDSMALL,200000.0,16.59,203826.0,17.0,3826.0,1.91
4,KKP-TQG,200000.0,8.15,209364.0,9.0,9364.0,4.68
5,KKP-SM-CAP,200000.0,14.22,208343.0,15.0,8343.0,4.17
6,TISCOFLEXP,200000.0,54.54,206853.0,56.0,6853.0,3.43
7,TSF-A,200000.0,75.66,198327.0,75.0,-1673.0,-0.84


In [18]:
dfPort=dfSummary[['BuyValue','SellValue','PL']].sum().to_frame('All Funds')
dfPort=dfPort.T
dfPort['PercentPL']=round( (dfPort['PL']/dfPort['BuyValue'] *100) ,0)
dfPort

,BuyValue,SellValue,PL,PercentPL
All Funds,1600000.0,1618630.0,18630.0,1.0


In [46]:
writer=pd.ExcelWriter(report_path,engine='xlsxwriter') 

dfPort.to_excel(writer,sheet_name='TotalPort',index=False)

dfSummary=dfSummary.sort_values(by=['Percent_PL'],ascending=False)
dfSummary.to_excel(writer, sheet_name="Summary",index=False)

for symbol,tranDF in dict_dfTran.items():
 tranDF=tranDF.drop(columns=['symbol'])   
 tranDF.to_excel(writer, sheet_name=symbol,index=False) 
writer.save()

writer.close()

print("Successfully")

C:\Users\pongthsa\AppData\Local\Temp\ipykernel_1796\886247830.py:11: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


Successfully


C:\Users\pongthsa\AppData\Roaming\Python\Python39\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
